In [33]:
from requests.auth import HTTPBasicAuth

import pandas as pd

import requests

def download_data(n=10000):
    r =requests.post('https://imunizacao-es.saude.gov.br/_search', auth=HTTPBasicAuth('imunizacao_public', 'qlto5t&7r_@+#Tlstigi'), json={"sizea":f"{n}"})
    if r.status_code != 200:
        raise "Error grabbing data"
    json_dict = r.json()
    data = json_dict['hits']['hits']
    print(len(data))
    return pd.DataFrame.from_records(data)
dfItem

<Response [200]>
10000


,_index,_type,_id,_score,_source
0,desc-imunizacao,_doc,bd5fb4c9-bb07-4ac0-a12c-169b01e4fcb9-i0b0,1.0,{'estabelecimento_razaoSocial': 'UNIMED BELO H...
1,desc-imunizacao,_doc,bcb7f6e2-7ca2-4af0-b3c0-ce0048a56e0e-i0b0,1.0,{'estabelecimento_razaoSocial': 'PREFEITURA DO...
2,desc-imunizacao,_doc,4dff5263-e80d-4361-9637-0021bf773355-i0b0,1.0,{'estabelecimento_razaoSocial': 'PM DE BAIXA G...
3,desc-imunizacao,_doc,d519e382-a573-42e1-8148-42d9b787986a-i0b0,1.0,{'estabelecimento_razaoSocial': 'MUNICIPIO DE ...
4,desc-imunizacao,_doc,44a46ce6-4f70-461a-b2d7-468e73caedd0-i0b0,1.0,{'estabelecimento_razaoSocial': 'PREFEITURA MU...
...,...,...,...,...,...
9995,desc-imunizacao,_doc,f2d170bf-91aa-45c3-82be-4b9986037cbd-i0b0,1.0,{'estabelecimento_razaoSocial': 'PREFEITURA MU...
9996,desc-imunizacao,_doc,9a786a8d-ba03-42e8-8aa2-bd199eef269e-i0b0,1.0,{'estabelecimento_razaoSocial': 'PREFEITURA MU...
9997,desc-imunizacao,_doc,36930960-40bb-4505-b9a1-4173b3f1f1de-i0b0,1.0,{'estabelecimento_razaoSocial': 'PREFEITURA DO...
9998,desc-imunizacao,_doc,361fd7bd-dd71-40d6-b5c7-8c9530e7b755-i0b0,1.0,{'estabelecimento_razaoSocial': 'PREFEITURA MU...


200